Step 1: Install dependencies

In [ ]:
!pip install pandas pyarrow boto3 trino


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.5 MB 11.2 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.5 MB 11.7 MB/s eta 0:00:01
   ---------------------- ----------------- 6.6/11.5 MB 10.9 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.5 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ------------- -------------------------- 8.9/25.7 MB 42.6 MB/s eta 0:00:01
   ----------------------------- ---------- 19.1/25.7 MB 46.5 MB/s eta 0:00:01
   ---------------------------------------  25.4/25.7 MB 44.8 MB/s eta 0:00:01
   ---------------------------------------- 25.7/25.7 MB 38.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   ----------------------------- ---------- 10.0/13.5 MB 47.8 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


📥 Step 2: Load and convert CSV → Parquet (locally)

In [1]:
import pandas as pd

df = pd.read_csv("spotify_dataset.csv")
df.to_parquet("spotify_dataset.parquet", index=False)

☁️ Step 3: Upload Parquet to S3 (in a folder)

🧱 Step 4: Create Parquet table in Trino over that S3 folder

In [14]:
# Comment out the next line if you don't want to upload a local file
# upload_local_parquet_to_s3()import trino
import pandas as pd
import os
from IPython.display import display, HTML

# Connect to Trino (previously known as PrestoSQL)
conn = trino.dbapi.connect(
    host='localhost',  # Use your Trino host
    port=8080,         # Default Trino port
    user='trino',      # Your Trino user
    catalog='s3',      # Your catalog name is s3, not hive
    schema='default'   # Starting with default schema
)

# Create a cursor
cursor = conn.cursor()

# Function to execute queries and print results
def execute_query(query, show_results=True):
    cursor.execute(query)
    if show_results and cursor.description:
        columns = [desc[0] for desc in cursor.description]
        results = cursor.fetchall()
        df = pd.DataFrame(results, columns=columns)
        print(f"Query executed successfully. Result rows: {len(df)}")
        if not df.empty and show_results:
            display(df)
        return df
    print("Query executed successfully. No results to display.")
    return None

# List available catalogs
print("Available catalogs:")
execute_query("SHOW CATALOGS")

# List schemas in the s3 catalog
print("\nAvailable schemas in s3 catalog:")
execute_query("SHOW SCHEMAS FROM s3")

Available catalogs:
Query executed successfully. Result rows: 2


,Catalog
0,s3
1,system



Available schemas in s3 catalog:
Query executed successfully. Result rows: 1


,Schema
0,information_schema


,Schema
0,information_schema


In [15]:
# Create a new schema if it doesn't exist
schema_name = "spotify"  # Your schema name 
try:
    execute_query(f"CREATE SCHEMA IF NOT EXISTS s3.{schema_name}", show_results=False)
    print(f"\nSchema '{schema_name}' created or already exists")
    
    # Verify the schema was created
    schemas_df = execute_query("SHOW SCHEMAS FROM s3")
    if schema_name in schemas_df[0].values:
        print(f"Confirmed: Schema '{schema_name}' exists in catalog 's3'")
    else:
        print(f"Warning: Schema '{schema_name}' was not found in the list of schemas")
        print("You may need manual configuration of your S3 connector to create schemas")
        
        # Fall back to default schema
        schema_name = "default"
        print(f"Falling back to using 's3.default' schema")
except Exception as e:
    print(f"Error creating schema: {e}")
    print("Falling back to using default schema")
    schema_name = "default"

Error creating schema: TrinoUserError(type=USER_ERROR, name=ALREADY_EXISTS, message="Schema already exists: 'spotify'", query_id=20250505_012637_00006_927zz)
Falling back to using default schema


In [18]:
schema_name = 'spotify'
try:
    execute_query(f"USE s3.{schema_name}")
    print(f"Successfully switched to schema s3.{schema_name}")
except Exception as e:
    print(f"Error switching to schema: {e}")
    print("Please check your Trino server configuration for S3 connector")

# Create the table for the Spotify dataset
# Note: Adjust the column definitions based on your actual parquet file structure
table_name = "spotify_tracks123"
create_table_query = f"""
CREATE TABLE IF NOT EXISTS s3.{schema_name}.{table_name} (
    id VARCHAR,
    name VARCHAR,
    popularity INTEGER,
    duration_ms BIGINT,
    explicit BOOLEAN,
    artists VARCHAR,
    id_artists VARCHAR,
    release_date VARCHAR,
    danceability DOUBLE,
    energy DOUBLE,
    key INTEGER,
    loudness DOUBLE,
    mode INTEGER,
    speechiness DOUBLE,
    acousticness DOUBLE,
    instrumentalness DOUBLE,
    liveness DOUBLE,
    valence DOUBLE,
    tempo DOUBLE
)
WITH (
    external_location = 's3a://spotify/spotify_dataset/',
    format = 'PARQUET'
)
"""
try:
    execute_query(create_table_query, show_results=False)
    print(f"\nTable '{table_name}' created or already exists")
except Exception as e:
    print(f"Error creating table: {e}")
    print("This might be due to permissions or S3 connector configuration issues")

Error switching to schema: TrinoUserError(type=USER_ERROR, name=NOT_FOUND, message="Schema does not exist: s3.spotify", query_id=20250505_012949_00011_927zz)
Please check your Trino server configuration for S3 connector
Error creating table: TrinoQueryError(type=INTERNAL_ERROR, name=GENERIC_INTERNAL_ERROR, message="The bucket is in this region: ap-northeast-2. Please use this region to retry the request (Service: Amazon S3; Status Code: 301; Error Code: 301 Moved Permanently; Request ID: 1GC6BWKYHPS40T70; S3 Extended Request ID: O9KamDW8JApc5u1mh/qKs4QOlDdGDEK74gdhLPcqomuSJwVxFSmHWAIf0Hlmuvv5ci+gE+h/Emo=; Proxy: null)", query_id=20250505_012949_00012_927zz)
This might be due to permissions or S3 connector configuration issues


In [ ]:
from trino.dbapi import connect
from trino.auth import BasicAuthentication

# Connect to Trino
conn = connect(
    host='localhost',  # or your docker host IP
    port=8080,
    user='admin',
    catalog='s3',
    schema='spotify',
    auth=BasicAuthentication("admin", ""),
    http_scheme='http'  # change to 'https' if applicable
)

cur = conn.cursor()

# Step 1: Create Schema (if not exists)
cur.execute("CREATE SCHEMA IF NOT EXISTS s3.spotify")

# Step 2: Create External Table over CSV in S3
cur.execute("""
CREATE TABLE IF NOT EXISTS s3.spotify.spotify_dataset (
    track_id VARCHAR,
    track_name VARCHAR,
    artist_name VARCHAR,
    album_name VARCHAR,
    duration_ms VARCHAR,
    popularity VARCHAR,
    danceability VARCHAR,
    energy VARCHAR,
    key VARCHAR,
    loudness VARCHAR,
    mode VARCHAR,
    speechiness VARCHAR,
    acousticness VARCHAR,
    instrumentalness VARCHAR,
    liveness VARCHAR,
    valence VARCHAR,
    tempo VARCHAR,
    time_signature VARCHAR
)
WITH (
    external_location = 's3a://spotify/spotify_dataset/',
    format = 'CSV',
    skip_header_line_count = 1
)
""")

# Step 3: Query the new table
cur.execute("SELECT * FROM s3.spotify.spotify_dataset LIMIT 5")
rows = cur.fetchall()

# Display rows
for row in rows:
    print(row)

['ABBA', "She's My Kind Of Girl", 'joy', '0.4476190476190476', 'pop', '2014', 'F Maj', '128', '-6.0', 'No', '31', '78', '56', '60', '3', '31', '7', '0']
['ABBA', 'Andante, Andante', 'love', '0.20222222222222216', 'pop', '1980', 'A# Maj', '102', '-10.72', 'No', '59', '36', '52', '38', '2', '7', '68', '0']
['ABBA', 'As Good As New', 'sadness', '0.3008807588075881', 'pop', '1979', 'E Maj', '139', '-5.7', 'No', '50', '78', '85', '97', '3', '8', '20', '2']
['ABBA', 'Bang', 'joy', '0.355', 'pop', '1975', 'F Maj', '132', '-3.0', 'No', '52', '76', '50', '89', '3', '32', '3', '0']
['ABBA', 'Bang-A-Boomerang', 'joy', '0.355', 'pop', '1975', 'F Maj', '132', '-3.0', 'No', '52', '76', '50', '89', '3', '32', '3', '0']


In [12]:
cur.execute("SHOW TABLES IN s3.spotify")
print(cur.fetchall())

cur.execute("SELECT * FROM s3.spotify.spotify_dataset LIMIT 5")
print(cur.fetchall())

[['playlist'], ['spotify_dataset']]
[['ABBA', "She's My Kind Of Girl", 'joy', '0.4476190476190476', 'pop', '2014', 'F Maj', '128', '-6.0', 'No', '31', '78', '56', '60', '3', '31', '7', '0'], ['ABBA', 'Andante, Andante', 'love', '0.20222222222222216', 'pop', '1980', 'A# Maj', '102', '-10.72', 'No', '59', '36', '52', '38', '2', '7', '68', '0'], ['ABBA', 'As Good As New', 'sadness', '0.3008807588075881', 'pop', '1979', 'E Maj', '139', '-5.7', 'No', '50', '78', '85', '97', '3', '8', '20', '2'], ['ABBA', 'Bang', 'joy', '0.355', 'pop', '1975', 'F Maj', '132', '-3.0', 'No', '52', '76', '50', '89', '3', '32', '3', '0'], ['ABBA', 'Bang-A-Boomerang', 'joy', '0.355', 'pop', '1975', 'F Maj', '132', '-3.0', 'No', '52', '76', '50', '89', '3', '32', '3', '0']]
